In [ ]:
!pip install -r requirements.txt

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import json
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection as model_selection
import sklearn.tree as tree
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import time
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import catboost
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split


In [ ]:
df_train = pd.read_csv('Training_clean.csv')
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

df_test = pd.read_csv('Test_clean.csv')
    


In [ ]:
df_train.head()

,id,seller,price,area_total,area_kitchen,area_living,floor,rooms,layout,ceiling,...,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating,price/area,distance,street
0,0,3,7139520.0,59.2,12.5,31.0,2,2,1,3,...,9,0,1,1,1,1,0,120600.000000,0.239348,Бунинские Луга ЖК
1,1,1,10500000.0,88.0,14.2,48.0,18,3,1,3,...,25,0,1,1,1,1,0,119318.181818,0.129713,улица Радужная
2,2,3,9019650.0,78.5,22.5,40.8,12,3,1,3,...,15,0,1,1,1,1,0,114900.000000,0.121183,Саларьево Парк ЖК
3,3,1,10500000.0,88.0,14.0,48.0,18,3,1,3,...,25,0,1,1,1,1,0,119318.181818,0.129713,улица Радужная
4,4,2,13900000.0,78.0,17.0,35.0,7,2,1,3,...,15,0,1,1,1,0,0,178205.128205,0.217762,улица Сервантеса


In [ ]:
#df_train['floor/stories'] = df_train.floor/df_train.stories
#df_test['floor/stories'] = df_test.floor/df_test.stories

#df_train['distance'] = np.sqrt((37.6 - df_train['longitude'])**2 + (55.75 - df_train['latitude'])**2)
#df_test['distance'] = np.sqrt((37.6 - df_test['longitude'])**2 + (55.75 - df_test['latitude'])**2)

#df_train.drop(['longitude', 'latitude'], axis=1, inplace=True)
#df_test.drop(['longitude', 'latitude'], axis=1, inplace=True)
df_train
df_test

,id,seller,area_total,area_kitchen,area_living,floor,rooms,layout,ceiling,bathrooms_shared,...,material,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating,distance,street
0,23285,3,71.4,13.16,29.44,2,1,1,3,1,...,2,20,1,1,1,1,1,0,0.052411,2-я Черногрязская улица
1,23286,1,54.0,4.00,50.00,4,1,1,3,1,...,2,15,0,1,1,0,0,0,0.216436,улица Сервантеса
2,23287,3,39.4,19.60,11.20,10,1,1,3,1,...,3,9,0,1,1,1,1,0,0.228879,Саларьево Парк ЖК
3,23288,3,60.9,15.68,55.56,5,2,1,3,1,...,2,17,1,1,0,0,1,3,0.217394,улица Александры Монаховой
4,23289,3,34.0,9.00,17.00,14,1,1,3,1,...,2,17,0,1,1,1,1,0,0.345584,улица 3-я Нововатутинская
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9932,33217,3,106.0,19.90,56.70,16,3,1,3,1,...,2,20,0,1,1,0,1,0,0.148935,улица Паршина
9933,33218,1,82.0,8.88,33.22,3,3,1,3,2,...,2,20,0,1,1,1,1,0,0.083756,Западный Порт ЖК
9934,33219,1,49.3,22.06,78.22,15,1,1,0,1,...,2,24,0,1,1,0,1,1,0.225713,Рублевское шоссе
9935,33220,1,38.8,10.50,15.10,14,1,1,3,1,...,1,14,1,1,1,0,1,0,0.062356,улица Архитектора Щусева


In [ ]:
#y1 = df_train['price']

X1_test = df_test.drop(['id', 'bathrooms_private', 'loggias', 'seller', 'bathrooms_shared', 'elevator_without', 'ceiling', 'material', 'constructed'], axis=1)

X1_test_area_total = X1_test.area_total.values # Save area total values from training set so that we can use feature engineering on price/area as measure
df_train['price/area'] = df_train['price']/df_train['area_total']
y1 = df_train['price/area'] # Don't drop area here because we need it for split

X1 = df_train.drop(['price', 'id', 'price/area', 'bathrooms_private', 'loggias', 'seller', 'bathrooms_shared', 'elevator_without', 'ceiling', 'material', 'constructed'], axis=1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X1, y1, test_size=0.2, random_state=1, shuffle=True)
X_train.shape, X_val.shape, y_train.shape,y_val.shape, X1_test.shape


((18589, 22), (4648, 22), (18589,), (4648,), (9937, 22))

In [ ]:
area_total_train = X_train.area_total.values
area_total_val = X_val.area_total.values
X_train = X_train.drop(['area_total'], axis=1)
X_val = X_val.drop(['area_total'], axis=1)
X_train.head()


#seller-0, rooms-3, layout-4, ceiling-5, bathrooms_shared-6, bathrooms_private-7, windows_court-8, windows_street-9, balconies-10, loggias-11, condition-12, phones-13, building_id-14, new-15, district-16, constructed-17, material-18, elevator_without-19, elevator_passenger-20, elevator_service-21, parking-22, garbage_chute-23, heating-24, street-25

,area_kitchen,area_living,floor,rooms,layout,windows_court,windows_street,balconies,condition,phones,...,new,district,stories,elevator_passenger,elevator_service,parking,garbage_chute,heating,distance,street
8161,16.1,28.2,3,2,1,1,0,0,1,1,...,1,11,14,1,1,0,1,0,0.209419,Скандинавия ЖК
10138,7.8,15.7,10,1,1,1,0,1,0,1,...,0,4,17,1,1,1,1,0,0.349929,улица Вертолетчиков
4845,6.0,30.0,3,2,1,0,1,1,1,1,...,0,6,9,1,0,1,1,0,0.141238,улица Островитянова
7189,12.4,33.0,18,2,1,1,1,1,0,2,...,0,2,20,1,1,1,1,0,0.103533,Нововладыкинский проезд
14728,14.4,27.1,14,2,1,1,0,1,1,1,...,1,8,20,1,1,1,1,0,0.108437,3-я Хорошевская улица


In [ ]:
#y1 = df_train.price.values
#X1 = df_train.drop(['id','price', 'street', 'address'], axis=1)
#X1_test = df_test.drop(['id', 'street', 'address'], axis=1)
categorical_features_indices = np.where(X_train.dtypes != np.float64)[0]
#categorical_features_indices = np.where(X1_test.dtypes != np.float64)[0]
#cat_feature_indices = np.array([0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25])

In [ ]:
#hyper parameter
n_estimators=[100,500,900,1100,1500]
max_depth=[2,3,5,10,15]
booster=["gbtree","gblinear"]
learning_rate="[0.05,0.1,0.15,0.20]"
min_child_weight=[1,2,3,4]
base_score=[0.25,0.5,0.75,1]

org_params = {
    'n_estimators': n_estimators,
    'learning_rate': learning_rate,
    "booster":booster,
    'eval_metric': 'RMSE',
    'use_best_model': True,
    'loss_function': 'RMSE',
    'max_depth': max_depth,
    "min_child_weight":min_child_weight,
    "base_score":base_score
}

In [ ]:
y_train.head()

8161     136249.667250
10138    157407.407407
4845     188888.888889
7189     176130.000000
14728    207950.000000
Name: price/area, dtype: float64

In [ ]:
#from catboost import CatBoostRegressor
#random_state=1, iterations=1500, depth=8, learning_rate=.1, loss_function='RMSE'
model = CatBoostRegressor(random_state=1, iterations=2000,cat_features = categorical_features_indices, depth=9, learning_rate=0.1, loss_function='RMSE')
model.fit(X_train, y_train, eval_set=(X_val, y_val), plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

1006:	learn: 20528.5783584	test: 60388.8714215	best: 60379.3466151 (1003)	total: 2m 45s	remaining: 2m 42s
1007:	learn: 20517.0841365	test: 60379.8954074	best: 60379.3466151 (1003)	total: 2m 45s	remaining: 2m 42s
1008:	learn: 20508.3210588	test: 60381.6257761	best: 60379.3466151 (1003)	total: 2m 45s	remaining: 2m 42s
1009:	learn: 20489.0168164	test: 60369.9983922	best: 60369.9983922 (1009)	total: 2m 45s	remaining: 2m 42s
1010:	learn: 20470.0317487	test: 60373.9057067	best: 60369.9983922 (1009)	total: 2m 45s	remaining: 2m 42s
1011:	learn: 20457.9374044	test: 60381.5081017	best: 60369.9983922 (1009)	total: 2m 45s	remaining: 2m 42s
1012:	learn: 20444.2001362	test: 60393.3098084	best: 60369.9983922 (1009)	total: 2m 46s	remaining: 2m 41s
1013:	learn: 20436.6303164	test: 60397.5632479	best: 60369.9983922 (1009)	total: 2m 46s	remaining: 2m 41s
1014:	learn: 20415.5401999	test: 60401.4686607	best: 60369.9983922 (1009)	total: 2m 46s	remaining: 2m 41s
1015:	learn: 20399.5874993	test: 60400.9801021

In [ ]:
final_df.head()

NameError: name 'final_df' is not defined

In [ ]:
print(model.score(X_train, y_train))
from sklearn.metrics import r2_score

y_pred = model.predict(X_val)
y_pred = y_pred.astype(float)
print(model.score(X_val, y_val)), print(r2_score(y_pred, model.predict(X_val)))

0.9572817800552783
0.8595282217218345
1.0


(None, None)

In [ ]:
df=pd.DataFrame({'Actual': y_val, 'Predicted':y_pred})
df

,Actual,Predicted
17722,200000.000000,191784.237390
10791,118300.000000,122979.171548
13285,182400.000000,180887.005855
14456,195876.288660,177614.247737
2329,91836.734694,100374.132811
...,...,...
5772,181500.000000,171044.128466
10324,323200.000000,316391.299027
17535,173333.333333,172005.601504
10757,496470.605744,374023.101101


In [ ]:
X1_test.head()

,seller,area_total,area_kitchen,area_living,floor,rooms,layout,ceiling,bathrooms_shared,bathrooms_private,...,material,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating,distance,street
0,3,71.4,13.16,29.44,2,1,1,3,1,1,...,2,20,1,1,1,1,1,0,0.052411,2-я Черногрязская улица
1,1,54.0,4.00,50.00,4,1,1,3,1,1,...,2,15,0,1,1,0,0,0,0.216436,улица Сервантеса
2,3,39.4,19.60,11.20,10,1,1,3,1,0,...,3,9,0,1,1,1,1,0,0.228879,Саларьево Парк ЖК
3,3,60.9,15.68,55.56,5,2,1,3,1,0,...,2,17,1,1,0,0,1,3,0.217394,улица Александры Монаховой
4,3,34.0,9.00,17.00,14,1,1,3,1,0,...,2,17,0,1,1,1,1,0,0.345584,улица 3-я Нововатутинская


In [ ]:
final_labels = model.predict(X1_test.drop(['area_total'], axis=1))
final_labels = final_labels.astype(float)
final_labels

array([361150.39855363, 192998.18246561, 163927.73336471, ...,
       220131.35845002, 264055.86376226, 109628.88359583])

In [ ]:

final_labels = final_labels * df_test.area_total.values
len(final_labels)

9937

In [ ]:
df_train.area_total.values

array([ 59.2,  88. ,  78.5, ...,  73.4, 100.8,  57.7])

In [ ]:
y_train_hat = model.predict(X_train) * area_total_train ## * edit
y_valid_hat = model.predict(X_val) * area_total_val ## * edit

y_train = y_train * area_total_train ## edit
y_val = y_val * area_total_val ## edit

#print(f'Train RMSLE: {root_mean_squared_log_error(y_true=y_train, y_pred=y_train_hat) :.4f}')
print(f'Valid RMSLE: {root_mean_squared_log_error(y_true=y_val, y_pred=y_valid_hat) :.4f}')

#Train RMSLE: 0.1283
#Valid RMSLE: 0.1761

#Train RMSLE: 0.0489
#Valid RMSLE: 0.1451

#Train RMSLE: 0.0071
#Valid RMSLE: 0.0187

Valid RMSLE: 0.1404


In [ ]:
submission = pd.DataFrame()
submission['id'] = df_test.id
submission['price_prediction'] = final_labels # Predict on non-nan entries
 # Fill missing entries with mean predictor
print(f'Generated {len(submission)} predictions')

# Export submission to csv with headers
#submission.to_csv('sample_submission.csv', index=False)

submission.to_csv('sample_submission9.csv', index=False)

Generated 9937 predictions


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=99c1e030-46bc-40fa-835c-0661eae0d488' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>